In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd

## Load data

In [3]:
network_type = "tl123_edges"

edas = {}

for i in range(1, 4, 1):
    eda_path = Path(f"analysis/{network_type}/{i}_eda.csv")
    eda = pd.read_csv(eda_path, index_col=0)
    eda = eda.reset_index().rename(columns={"index": "node"})
    eda["cipa"] = i
    edas[f"{i}"] = eda


In [4]:
# Merge DataFrames
merged_df = pd.concat([edas["1"], edas["2"], edas["3"]], keys=['snapshot1', 'snapshot2', 'snapshot3'])

In [5]:
# Select only numerical columns for grouping and mean calculation
numerical_columns = merged_df.select_dtypes(include='number').columns
average_values = merged_df[numerical_columns].groupby('node').mean()

In [6]:
average_values

,metric_TLuseoutofclass,metric_postsojournOPI,metric_presojournOPI,metric_level gained,metric_FLcumulativecompetence,metric_general_cumulativecompet,context_learningoutofclassminday,psycho_motivationdegree,psycho_proficiencyingroup_BAL1,interact_classmates,...,degree_weighted,degree,betweenness_weighted,betweenness,closeness,pagerank_weighhted,pagerank,voterank,clustering_coefficient,cipa
node,,,,,,,,,,,,,,,,,,,,,
1,0.630702,0.727273,0.545455,0.181818,15.0,21.0,0.069444,1.000000,0.333333,0.600000,...,1.733333,2.000000,0.000000,0.000000,0.056837,0.018107,0.016333,15.000000,0.500000,2.0
2,0.934837,0.636364,0.363636,0.272727,6.0,12.0,0.125000,0.800000,0.666667,0.543333,...,4.866667,8.333333,0.040001,0.039809,0.152953,0.043047,0.049067,10.333333,0.311111,2.0
3,1.000000,0.636364,0.363636,0.272727,3.0,9.0,0.090278,0.800000,0.666667,0.483333,...,3.266667,5.666667,0.035217,0.034842,0.058389,0.017397,0.018825,1.000000,0.034921,2.0
4,0.178947,0.545455,0.454545,0.090909,6.0,12.0,0.093750,0.700000,1.000000,0.875000,...,2.200000,3.000000,0.000000,0.000000,0.000000,0.010286,0.010267,5.000000,0.000000,1.5
5,0.409428,0.636364,0.636364,0.000000,3.0,9.0,0.152778,0.933333,0.666667,0.666667,...,1.733333,2.666667,0.005580,0.005580,0.049875,0.021902,0.023889,13.000000,0.277778,2.0
6,0.361111,0.545455,0.454545,0.090909,5.0,11.0,0.090278,0.866667,0.777778,0.566667,...,2.666667,3.666667,0.000000,0.000000,0.132836,0.040565,0.037965,NaN,0.847222,2.0
7,0.388889,0.636364,0.454545,0.181818,8.0,14.0,0.185185,1.000000,0.555556,0.650000,...,2.266667,3.333333,0.031782,0.031657,0.084915,0.029990,0.028684,5.000000,0.076923,2.0
8,0.286842,0.636364,0.454545,0.181818,6.0,12.0,0.049769,0.666667,0.777778,0.650000,...,1.800000,2.666667,0.027528,0.027528,0.047828,0.015176,0.015059,3.000000,0.133333,2.0
9,0.973447,0.727273,0.363636,0.363636,8.5,14.5,0.143519,0.666667,0.666667,0.600000,...,1.400000,2.333333,0.001217,0.001217,0.052175,0.020318,0.019632,12.000000,0.000000,2.0
